## SKIP TO V3 at the bottom

## V1 V2 are out dated

## V1 -  CORTEX MULTIMODAL IMAGES FOR CALLAWAY

In [ ]:
USE DATABASE CALLAWAY_IMG_COMPLETE;
USE SCHEMA PUBLIC;
SELECT SNOWFLAKE.CORTEX.COMPLETE('claude-3-5-sonnet', 
    'Summarize the golf club image and try to detect the club type, name, brand, model, shaft type, shaft flex, loft, hand (left or righty), classification notes, condition assessment and estimated price range',
    TO_FILE('@IMG_STAGE', 'CLWY_BB_1_MANUAL.jpg'));

In [ ]:
USE DATABASE CALLAWAY_IMG_COMPLETE;
USE SCHEMA PUBLIC;
SELECT SNOWFLAKE.CORTEX.COMPLETE('claude-3-5-sonnet', 
    'Summarize the golf club image and try to detect the club type, name, brand, model, shaft type, shaft flex, loft, hand (left or righty), classification notes, condition assessment and estimated price range. Can you also find out what year or era the club may be from? ',
    TO_FILE('@IMG_STAGE', 'GC10.jpg'));

In [ ]:
USE DATABASE CALLAWAY_IMG_COMPLETE;
USE SCHEMA PUBLIC;
SELECT SNOWFLAKE.CORTEX.COMPLETE('pixtral-large', 
    'Summarize the golf club image and try to detect the club type, name, brand, model, shaft type, shaft flex, loft, hand (left or righty), classification notes, condition assessment and estimated price range. Can you also find out what year or era the club may be from?',
    TO_FILE('@IMG_STAGE', 'GC10.jpg'));

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE('claude-3-5-sonnet','Please analyze this golf club image and provide the following:
    "club_type": "driver/iron/wedge/putter/hybrid/etc",
    "club_name": "full name",
    "brand": "manufacturer name",
    "model": "specific model",
    "year": "estimated year or range as number",
    "club_category": "game improvement/players/etc",
    "shaft_type": "steel/graphite/etc",
    "shaft_flex": "regular/stiff/extra-stiff/etc",
    "loft": "degree as number only",
    "hand": "left/right",
    "confidence_score": "number between 0 and 1",
    "classification_notes": "detailed observations about distinguishing features",
    "condition_assessment": "excellent/good/fair/poor",
    "estimated_price_range": "low-high in dollars",
    "features": {
        "adjustable": "yes/no",
        "forged": "yes/no",
        "cavity_back": "yes/no",
        "special_technology": "brief description"}
    Provide high confidence answers when possible. If you cannot determine a value, use null instead of 
    guessing.',

    TO_FILE('@IMG_STAGE', 'GC10.jpg'));

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE('claude-4-sonnet','Please analyze this golf club image and provide the following:
    "club_type": "driver/iron/wedge/putter/hybrid/etc",
    "club_name": "full name",
    "brand": "manufacturer name",
    "model": "specific model",
    "year": "estimated year or range as number",
    "club_category": "game improvement/players/etc",
    "shaft_type": "steel/graphite/etc",
    "shaft_flex": "regular/stiff/extra-stiff/etc",
    "loft": "degree as number only",
    "hand": "left/right",
    "confidence_score": "number between 0 and 1",
    "classification_notes": "detailed observations about distinguishing features",
    "condition_assessment": "excellent/good/fair/poor",
    "estimated_price_range": "low-high in dollars",
    "features": {
        "adjustable": "yes/no",
        "forged": "yes/no",
        "cavity_back": "yes/no",
        "special_technology": "brief description"}
    Provide high confidence answers when possible. If you cannot determine a value, use null instead of 
    guessing.',

    TO_FILE('@IMG_STAGE', 'GC10.jpg'));

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE('claude-4-sonnet', 'Where is Snowflake HQ and what is the latest stock price');

In [ ]:
--SELECT TO_FILE('@IMG_STAGE', 'CLWY_BB_1.jpg');

SELECT TO_FILE('@IMG_STAGE', 'GC_15.png');

In [ ]:
LS @IMG_STAGE;

In [ ]:
select * from CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS order by analysis_timestamp DESC ;

In [ ]:

-- NOTE THIS IS NOT USING THE NEW FILE / UNSTRUCTERED FIELD FOR IMAGES
create or replace TABLE CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS (
	ANALYSIS_ID NUMBER(38,0) NOT NULL autoincrement start 1 increment 1 noorder,
	IMAGE_ID VARCHAR(500),
	ANALYSIS_TIMESTAMP TIMESTAMP_LTZ(9) DEFAULT CURRENT_TIMESTAMP(),
	CLUB_TYPE VARCHAR(500),
	CLUB_NAME VARCHAR(500),
	BRAND VARCHAR(500),
	MODEL VARCHAR(500),
	YEAR VARCHAR(500),
	CLUB_CATEGORY VARCHAR(500),
	SHAFT_TYPE VARCHAR(500),
	SHAFT_FLEX VARCHAR(500),
	LOFT FLOAT,
	HAND VARCHAR(500),
	CONFIDENCE_SCORE FLOAT,
	CLASSIFICATION_NOTES VARCHAR(16777216),
	CONDITION_ASSESSMENT VARCHAR(500),
	ESTIMATED_PRICE_RANGE VARCHAR(500),
	FEATURES VARIANT,
	CREATED_BY VARCHAR(500) DEFAULT CURRENT_USER(),
	CREATED_AT TIMESTAMP_LTZ(9) DEFAULT CURRENT_TIMESTAMP(),
	UPDATED_AT TIMESTAMP_LTZ(9) DEFAULT CURRENT_TIMESTAMP(),
	primary key (ANALYSIS_ID)
);

In [ ]:
ALTER TABLE CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS ALTER (
  IMAGE_ID SET DATA TYPE VARCHAR(500),
  CLUB_TYPE SET DATA TYPE VARCHAR(500),
  CLUB_NAME SET DATA TYPE VARCHAR(500),
  BRAND SET DATA TYPE VARCHAR(500),
  MODEL SET DATA TYPE VARCHAR(500),
  CLUB_CATEGORY SET DATA TYPE VARCHAR(500),
  SHAFT_TYPE SET DATA TYPE VARCHAR(500),
  SHAFT_FLEX SET DATA TYPE VARCHAR(500),
  HAND SET DATA TYPE VARCHAR(500),
  CONDITION_ASSESSMENT SET DATA TYPE VARCHAR(500),
  ESTIMATED_PRICE_RANGE SET DATA TYPE VARCHAR(500),
  CREATED_BY SET DATA TYPE VARCHAR(500)
);

In [ ]:
truncate table CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS;

In [ ]:
DELETE FROM CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS
WHERE  analysis_id = 204;

# *NEW* - USE FILE TYPE

## This Demo will go over uploading an IMG directly to Snowflake and processing the data using `AI SQL` Functions

In [ ]:
USE ROLE ACCOUNTADMIN;

In [ ]:
import streamlit as st
import pandas as pd
import snowbooks_extras

In [ ]:
create or replace TABLE CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS_FILE (
	ANALYSIS_ID NUMBER(38,0) NOT NULL autoincrement start 1 increment 1 noorder,
	IMAGE_ID VARCHAR(255),
    img FILE,
	ANALYSIS_TIMESTAMP TIMESTAMP_LTZ(9) DEFAULT CURRENT_TIMESTAMP(),
	CLUB_TYPE VARCHAR(50),
	CLUB_NAME VARCHAR(255),
	BRAND VARCHAR(50),
	MODEL VARCHAR(100),
	YEAR NUMBER(4,0),
	CLUB_CATEGORY VARCHAR(50),
	SHAFT_TYPE VARCHAR(50),
	SHAFT_FLEX VARCHAR(20),
	LOFT FLOAT,
	HAND VARCHAR(10),
	CONFIDENCE_SCORE FLOAT,
	CLASSIFICATION_NOTES VARCHAR(16777216),
	CONDITION_ASSESSMENT VARCHAR(50),
	ESTIMATED_PRICE_RANGE VARCHAR(50),
	FEATURES VARIANT,
	CREATED_BY VARCHAR(100) DEFAULT CURRENT_USER(),
	CREATED_AT TIMESTAMP_LTZ(9) DEFAULT CURRENT_TIMESTAMP(),
	UPDATED_AT TIMESTAMP_LTZ(9) DEFAULT CURRENT_TIMESTAMP(),
	primary key (ANALYSIS_ID)
);

In [ ]:
/*
CREATE TABLE CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS_FILE (
    -- Image as first column
    IMG FILE,
    
    -- Basic identification
    IMAGE_ID VARCHAR(255),
    
    -- Club details
    CLUB_TYPE VARCHAR(50),
    CLUB_NAME VARCHAR(255),
    BRAND VARCHAR(50),
    MODEL VARCHAR(100),
    YEAR NUMBER(4,0),
    CLUB_CATEGORY VARCHAR(50),
    SHAFT_TYPE VARCHAR(50),
    SHAFT_FLEX VARCHAR(20),
    LOFT FLOAT,
    HAND VARCHAR(10),
    
    -- Analysis fields
    CONFIDENCE_SCORE FLOAT,
    CLASSIFICATION_NOTES VARCHAR(16777216),
    CONDITION_ASSESSMENT VARCHAR(50),
    ESTIMATED_PRICE_RANGE VARCHAR(50),
    FEATURES VARIANT,
    ANALYSIS_TIMESTAMP TIMESTAMP_LTZ(9) DEFAULT CURRENT_TIMESTAMP(),
    
    -- File metadata from DIRECTORY()
    RELATIVE_PATH VARCHAR(16777216),
    FILE_URL VARCHAR(16777216),
    SIZE NUMBER,
    LAST_MODIFIED TIMESTAMP_LTZ,
    MD5 VARCHAR(32),
    
    -- Tracking fields
    CREATED_BY VARCHAR(100) DEFAULT CURRENT_USER(),
    CREATED_AT TIMESTAMP_LTZ(9) DEFAULT CURRENT_TIMESTAMP(),
    UPDATED_AT TIMESTAMP_LTZ(9) DEFAULT CURRENT_TIMESTAMP(),

    -- AI Model
    AI_MODEL VARCHAR(255)
);
*/

In [ ]:
ALTER TABLE CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS_FILE
ADD COLUMN AI_MODEL VARCHAR(255);

In [ ]:
ls @CALLAWAY_IMG_COMPLETE.PUBLIC.IMG_STAGE;

In [ ]:
INSERT INTO CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS_FILE (
    IMG,
    IMAGE_ID,
    CLUB_TYPE,  -- Changed to CLUB_TYPE as your table likely doesn't have RELATIVE_PATH
    CLUB_NAME,  -- Changed to CLUB_NAME as your table likely doesn't have FILE_URL
    BRAND,      -- Changed to BRAND as your table likely doesn't have SIZE
    MODEL,      -- Changed to MODEL as your table likely doesn't have LAST_MODIFIED
    YEAR        -- Changed to YEAR as your table likely doesn't have MD5
)
SELECT 
    TO_FILE(file_url) AS IMG,
    -- Extract filename without extension as IMAGE_ID
    REGEXP_REPLACE(
        SPLIT_PART(relative_path, '/', -1),
        '\\.jpg$|\\.jpeg$|\\.png$|\\.gif$', ''
    ) AS IMAGE_ID,
    NULL AS CLUB_TYPE,
    NULL AS CLUB_NAME,
    NULL AS BRAND,
    NULL AS MODEL,
    NULL AS YEAR
FROM 
    DIRECTORY(@CALLAWAY_IMG_COMPLETE.PUBLIC.IMG_STAGE)
WHERE 
    REGEXP_LIKE(relative_path, '.*\\.(jpg|jpeg|png|gif)$', 'i');

In [ ]:
SELECT 
  relative_path,
  file_url,
  size,
  last_modified
FROM 
  DIRECTORY(@CALLAWAY_IMG_COMPLETE.PUBLIC.IMG_STAGE)
ORDER BY
  last_modified DESC;

In [ ]:
INSERT INTO CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS_FILE (
    IMAGE_ID,
    img
)
SELECT
    -- Extract filename and remove extension
    REGEXP_REPLACE(
        SPLIT_PART(relative_path, '/', -1),    -- Get last part after splitting by '/'
        '\\.jpg$|\\.jpeg$|\\.png$', ''        -- Remove file extension (note escaped dots)
    ) AS IMAGE_ID,
    TO_FILE(file_url) AS img
FROM 
    DIRECTORY(@CALLAWAY_IMG_COMPLETE.PUBLIC.IMG_STAGE)
WHERE 
    REGEXP_LIKE(relative_path, '.*\\.(jpg|jpeg|png)$', 'i');  -- Using 'i' as separate parameter for case insensitivity

In [ ]:
SELECT * FROM CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS_FILE ORDER BY CREATED_AT DESC;

In [ ]:
CREATE OR REPLACE TABLE CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS_FILTERED AS
SELECT 
    -- Club details
    CLUB_TYPE,
    CLUB_NAME,
    BRAND,
    MODEL,
    YEAR,
    CLUB_CATEGORY,
    SHAFT_TYPE,
    SHAFT_FLEX,
    LOFT,
    HAND,
    
    -- Analysis fields
    CONFIDENCE_SCORE,
    CLASSIFICATION_NOTES,
    CONDITION_ASSESSMENT,
    ESTIMATED_PRICE_RANGE,
    FEATURES,
    ANALYSIS_TIMESTAMP,
    
    -- Tracking fields
    CREATED_BY,
    CREATED_AT,
    UPDATED_AT,
    
    -- AI Model
    AI_MODEL
FROM CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS_FILE
WHERE CLUB_TYPE IS NOT NULL;

## AI FILTER - Classifies free-form prompt inputs into a boolean. Currently supports both text and image filtering

#### `How does it work?` 
AI_FILTER is Snowflake's smart filtering function that lets you search your text data using natural language. Rather than writing complex SQL conditions, you simply describe what you're looking for, and it returns the matching records - like having a conversation with your database instead of writing code.

i.e. Scratches, wear, used for example

Reference https://docs.snowflake.com/LIMITEDACCESS/snowflake-cortex/ai_filter

In [ ]:
-- Find reviews mentioning scratches, wear, used etc
SELECT club_type, club_name, brand, model, year, confidence_score, classification_notes, created_at
FROM CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS_FILTERED
WHERE AI_FILTER('The notes mentions scratches or wear or used: ' || classification_notes)
ORDER BY created_at DESC;

## AI AGG - Reduces a column of text data using a natural language task description.

#### `How Does it work? `
AI_AGG is Snowflake's intelligent aggregation function that acts like a data analyst in a box. It groups your text data and then uses AI to analyze patterns, extract insights, and generate summaries across each group. Think of it as automatically distilling thousands of comments into actionable intelligence without manually reading each one.

Reference: https://docs.snowflake.com/LIMITEDACCESS/snowflake-cortex/ai_agg

In [ ]:
-- Golf Club Analysis By Club Type and Brand
--- The SUMMARIZE_AGG function will automatically condense the text data into meaningful summaries without requiring an explicit prompt, making it simpler to use than AI_AGG

SELECT  CLUB_TYPE,
        BRAND,
        AI_AGG('Club Type: ' || CLUB_TYPE || 
               '\n\nBrand: ' || BRAND || 
               '\n\nAnalysis Notes: ' || CLASSIFICATION_NOTES, 
               'Analyze these golf club assessment notes for this specific club type and brand. Identify the top strengths and weaknesses mentioned in the classification notes. Provide 2-3 specific insights about common conditions, features, or issues found across this club type.') as analysis
FROM CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS_FILTERED
WHERE CLASSIFICATION_NOTES IS NOT NULL
  AND TRIM(CLASSIFICATION_NOTES) <> ''
GROUP BY CLUB_TYPE, BRAND;

## SUMMARIZE AGG - Summarizes a column of text data.

#### `How does it work?` 
SUMMARIZE_AGG is Snowflake's automatic text condensing function that turns large volumes of text into concise, readable summaries. Unlike its cousin AI_AGG, it doesn't require custom prompts - just point it at your text data, group as needed, and it automatically distills key points while maintaining the essential meaning across each group.

Reference: https://docs.snowflake.com/LIMITEDACCESS/snowflake-cortex/ai_summarize_agg

In [ ]:
-- Golf Club Condition Analysis
-- Generate summaries by condition category to understand different club assessments
SELECT
    CONDITION_ASSESSMENT,
    SUMMARIZE_AGG(CLASSIFICATION_NOTES) AS condition_summary
FROM CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS_FILTERED
WHERE CLASSIFICATION_NOTES IS NOT NULL
  AND TRIM(CLASSIFICATION_NOTES) <> ''
GROUP BY CONDITION_ASSESSMENT
ORDER BY CONDITION_ASSESSMENT;

## AI CLASSIFY - Classifies text or images into categories that you specify.

#### `How does it work? `
AI_CLASSIFY is Snowflake's intelligent categorization function that automatically sorts your text data into custom categories you define. Simply provide your text and a list of categories, and it assigns each record to the most relevant bucket - turning unstructured comments into structured, analyzable data without manual tagging or complex rule creation

Reference: https://docs.snowflake.com/LIMITEDACCESS/snowflake-cortex/ai_classify

In [ ]:
-- Golf Club Classification Analysis
-- Categorize club assessment notes into specific issue/feature types
SELECT  *,
        AI_CLASSIFY(CLASSIFICATION_NOTES,
        ['Club Head Condition', 'Shaft Performance', 'Grip Quality', 'Face Damage', 'Cosmetic Issues', 'Performance Features', 'Authenticity Verification', 'Value Assessment']) as analysis
FROM CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS_FILTERED
WHERE CLASSIFICATION_NOTES IS NOT NULL
  AND TRIM(CLASSIFICATION_NOTES) <> '';

## AI ENTITY_SENTIMENT

Cortex AI ENTITY_SENTIMENT captures a spectrum of customer opinions beyond overall positive, negative, and neutral buckets. For aspect-based sentiment analysis, specify the content (such as a customer comment or a review) and the entities (aspects or categories) for which you want to analyze sentiment. ENTITY_SENTIMENT returns sentiment scores for each entity as well as an overall sentiment score.


Reference: https://docs.snowflake.com/en/user-guide/snowflake-cortex/sentiment

In [ ]:
-- Golf Club Sentiment Analysis
-- Analyze sentiment in club assessment notes to identify positive and negative trends
SELECT  
    CLUB_TYPE,
    BRAND,
    MODEL,
    CLASSIFICATION_NOTES,
    SENTIMENT(CLASSIFICATION_NOTES) as sentiment_category,
    SENTIMENT_SCORE(CLASSIFICATION_NOTES) as sentiment_score
FROM CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS_FILTERED
WHERE CLASSIFICATION_NOTES IS NOT NULL
  AND TRIM(CLASSIFICATION_NOTES) <> ''
ORDER BY sentiment_score DESC;

# V3 *NEW* - USE FILE TYPE & MULTI IMAGE SUPPORT W/CLAUDE


In [ ]:
# Define image in a stage and read the file
image=session.file.get_stream('@CALLAWAY_IMG_COMPLETE.PUBLIC.IMG_STAGE/ERD.jpg' , decompress=False).read() 

# Display the image
st.image(image, width=1200)

In [ ]:
/*
-- Enhanced main table with all new fields for comprehensive golf club analysis
CREATE OR REPLACE TABLE CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS_FILE_V3 (
    -- Basic identification
    IMAGE_ID VARCHAR(255) PRIMARY KEY,
    
    -- Club details (existing fields)
    CLUB_TYPE VARCHAR(50),
    CLUB_NAME VARCHAR(255),
    BRAND VARCHAR(50),
    MODEL VARCHAR(100),
    YEAR NUMBER(4,0),
    CLUB_CATEGORY VARCHAR(50),
    SHAFT_TYPE VARCHAR(50),
    SHAFT_FLEX VARCHAR(20),
    SHAFT_LABEL VARCHAR(100),                    -- Moved from enhanced
    LOFT FLOAT,
    HAND VARCHAR(10),
    
    -- *** NEW ENHANCED CLUB SPECIFICATION FIELDS ***
    CLUB_SUB_TYPE VARCHAR(50),                   -- NEW: Iron type, wedge type, etc.
    SET_COMPOSITION VARCHAR(100),                -- NEW: Set information
    LIE_ANGLE FLOAT,                            -- NEW: Lie angle in degrees
    FACE_ANGLE VARCHAR(20),                     -- NEW: Open/Closed/Neutral
    BOUNCE_ANGLE FLOAT,                         -- NEW: Bounce angle for wedges
    GRIND_TYPE VARCHAR(50),                     -- NEW: Sole grind type
    MODEL_DESIGNATION VARCHAR(100),             -- NEW: Specific model variants
    SHAFT_LENGTH_INCHES FLOAT,                  -- NEW: Shaft length
    
    -- *** NEW DETAILED CONDITION ASSESSMENT FIELDS ***
    OVERALL_GRADE NUMBER(3,1),                  -- NEW: Overall grade 1-10
    FACE_SOLE_WEAR_GRADE VARCHAR(20),          -- NEW: Face/sole wear grade
    FACE_SOLE_WEAR_DESCRIPTION VARCHAR(500),   -- NEW: Detailed wear description
    SCRATCHES_GRADE VARCHAR(20),               -- NEW: Scratch severity grade
    SCRATCHES_DESCRIPTION VARCHAR(500),        -- NEW: Scratch details
    SCRATCHES_LOCATIONS ARRAY,                 -- NEW: Array of scratch locations
    PAINT_CHIPS_GRADE VARCHAR(20),             -- NEW: Paint chip grade
    PAINT_CHIPS_DESCRIPTION VARCHAR(500),      -- NEW: Paint chip details
    PAINT_CHIPS_LOCATIONS ARRAY,              -- NEW: Array of paint chip locations
    PUTTER_PAINT_WEAR_GRADE VARCHAR(20),       -- NEW: Putter-specific paint wear
    GRIP_CONDITION VARCHAR(50),                -- NEW: Grip condition assessment
    
    -- *** NEW MARKET VALUATION FIELDS ***
    RETAIL_PRICE NUMBER(8,2),                  -- NEW: Current retail price
    TRADE_IN_VALUE NUMBER(8,2),                -- NEW: Trade-in/resale value
    MARKET_DEMAND VARCHAR(20),                 -- NEW: Market demand level
    
    -- *** NEW TECHNOLOGY AND FEATURES FIELDS ***
    TECHNOLOGY_TAGS ARRAY,                     -- NEW: Array of technology features
    SPECIAL_DESIGNATIONS ARRAY,                -- NEW: Array of special designations
    
    -- Analysis fields (existing)
    CONFIDENCE_SCORE FLOAT,
    CLASSIFICATION_NOTES VARCHAR(16777216),
    CONDITION_ASSESSMENT VARCHAR(50),
    ESTIMATED_PRICE_RANGE VARCHAR(50),
    FEATURES VARIANT,
    ANALYSIS_TIMESTAMP TIMESTAMP_LTZ(9) DEFAULT CURRENT_TIMESTAMP(),
    
    -- File metadata from DIRECTORY()
    RELATIVE_PATH VARCHAR(16777216),
    FILE_URL VARCHAR(16777216),
    SIZE NUMBER,
    LAST_MODIFIED TIMESTAMP_LTZ,
    MD5 VARCHAR(32),
    
    -- Tracking fields
    CREATED_BY VARCHAR(100) DEFAULT CURRENT_USER(),
    CREATED_AT TIMESTAMP_LTZ(9) DEFAULT CURRENT_TIMESTAMP(),
    UPDATED_AT TIMESTAMP_LTZ(9) DEFAULT CURRENT_TIMESTAMP(),
    AI_MODEL VARCHAR(255)
);

-- *** NEW TABLE FOR MULTI-IMAGE REFERENCES V3 ***
CREATE OR REPLACE TABLE CALLAWAY_IMG_COMPLETE.PUBLIC.IMAGE_REFERENCES_V3 (
    IMG FILE, 
    REFERENCE_ID VARCHAR(255) PRIMARY KEY DEFAULT (UUID_STRING()),
    PRIMARY_IMAGE_ID VARCHAR(255),
    REFERENCE_IMAGE_PATH VARCHAR(500),
    IMAGE_SEQUENCE NUMBER(3,0),
    REFERENCE_TYPE VARCHAR(50), -- 'MULTI_ANGLE', 'SET_MEMBER', 'COMPARISON'
    CREATED_AT TIMESTAMP_LTZ(9) DEFAULT CURRENT_TIMESTAMP(),
    FOREIGN KEY (PRIMARY_IMAGE_ID) REFERENCES CALLAWAY_CLUBS_FILE_V3(IMAGE_ID)
);

-- *** NEW TABLE FOR DETAILED DEFECT OBSERVATIONS V3 ***
CREATE OR REPLACE TABLE CALLAWAY_IMG_COMPLETE.PUBLIC.CLUB_DEFECT_OBSERVATIONS_V3 (
    OBSERVATION_ID VARCHAR(255) PRIMARY KEY,
    IMAGE_ID VARCHAR(600),
    DEFECT_TYPE VARCHAR(600),
    DEFECT_LOCATION VARCHAR(600),
    DEFECT_SIZE VARCHAR(600),
    DEFECT_LENGTH_MM FLOAT,
    DEFECT_WIDTH_MM FLOAT,
    DEFECT_DEPTH VARCHAR(600),
    IMPACT_ON_PERFORMANCE VARCHAR(600),
    CREATED_AT TIMESTAMP_LTZ(9) DEFAULT CURRENT_TIMESTAMP(),
    FOREIGN KEY (IMAGE_ID) REFERENCES CALLAWAY_CLUBS_FILE_V3(IMAGE_ID)
);

# 💰 Snowflake Cortex AI Cost Analysis
## 🚀 Executive Summary
> **Bottom Line:** Processing golf club images with Snowflake Cortex AI costs less than **1 cent per image!** 
---
I did 7 CORTEX.COMPLETE MULTIMODAL (see image for the count) runs today with 57 total images (also in same pic), here's the cost breakdown:
- 0.1515 Total Credits (see attached image of a streamlit app which monitors AI cost looking at SNOWFLAKE.ACCOUNT_USAGE.CORTEX_FUNCTIONS_USAGE_HISTORY)
- 0.1515 credits ÷ 7 queries = 0.0216 credits per query
- 0.0216 credits × $2.40 / credit = $0.052 per query (about 5.2 cents per query)
- 0.1515 credits × $2.40 / credit = $0.36 (36 cents total)
- Average credits per query = 0.1515 ÷ 7 = 0.0216 credits
- Cost per image = $0.36 ÷ 57 images = $0.006 (about 0.6 cents per image)

In [ ]:
USE ROLE ACCOUNTADMIN;

In [ ]:
import streamlit as st
import pandas as pd
import snowbooks_extras
import streamlit as st
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
SELECT * from CALLAWAY_IMG_COMPLETE.PUBLIC.IMAGE_REFERENCES_V3;

In [ ]:
SELECT COUNT(*) as TOTAL_QUERIES_TODAY
FROM CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS_FILE_V3
WHERE DATE(CREATED_AT) = CURRENT_DATE();

In [ ]:
SELECT COUNT(*) as TOTAL_IMAGE_REFERENCES_TODAY
FROM CALLAWAY_IMG_COMPLETE.PUBLIC.IMAGE_REFERENCES_V3
WHERE DATE(CREATED_AT) = CURRENT_DATE();

In [ ]:
SELECT * FROM CALLAWAY_IMG_COMPLETE.PUBLIC.CLUB_DEFECT_OBSERVATIONS_V3
WHERE DATE(CREATED_AT) = CURRENT_DATE()
ORDER BY CREATED_AT DESC;

In [ ]:
SELECT * FROM CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS_FILE_V3
ORDER BY ANALYSIS_TIMESTAMP DESC;

In [ ]:
SELECT * FROM  CALLAWAY_IMG_COMPLETE.PUBLIC.IMAGE_REFERENCES_V3 
WHERE DATE(CREATED_AT) = CURRENT_DATE()
ORDER BY CREATED_AT DESC;

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'claude-4-sonnet',
    PROMPT('Analyze image {0}, image {1}, image {2}, image {3}, image {4}, image {5}, image {6}, image {7} of the same golf club comprehensively and extract ALL possible information. Return ONLY valid JSON with this complete structure:
    [
    "club_type": "driver/fairway_wood/hybrid/iron/wedge/putter",
    "club_name": "full descriptive name",
    "brand": "manufacturer name", 
    "model": "specific model name",
    "year": 2023,
    "club_category": "game_improvement/players/distance/forgiveness",
    "shaft_type": "steel/graphite/hybrid",
    "shaft_flex": "extra_stiff/stiff/regular/senior/ladies",
    "shaft_label": "shaft brand and model if visible",
    "loft": 10.5,
    "hand": "right/left",
    "club_sub_type": "cavity_back/blade/mallet/etc",
    "set_composition": "individual/set_member/part_of_set",
    "lie_angle": 59.0,
    "face_angle": "neutral/open/closed",
    "bounce_angle": 12.0,
    "grind_type": "sole_grind_if_wedge",
    "model_designation": "specific_variant_or_edition",
    "shaft_length_inches": 45.0,
    "overall_grade": 8.5,
    "face_sole_wear_grade": "excellent/very_good/good/fair/poor",
    "face_sole_wear_description": "detailed description of face and sole condition including groove wear, impact marks, etc",
    "scratches_grade": "none/minor/moderate/severe",
    "scratches_description": "detailed description of all visible scratches, their location and severity",
    "scratches_locations": ["face", "sole", "crown", "back"],
    "paint_chips_grade": "none/minor/moderate/severe", 
    "paint_chips_description": "detailed description of paint/finish condition",
    "paint_chips_locations": ["crown", "sole", "back"],
    "putter_paint_wear_grade": "excellent/good/fair/poor",
    "grip_condition": "excellent/very_good/good/fair/poor/needs_replacement",
    "retail_price": 399.99,
    "trade_in_value": 150.00,
    "market_demand": "high/medium/low",
    "technology_tags": ["technology_1", "technology_2", "specific_features"],
    "special_designations": ["tour_issue", "limited_edition", "custom", "prototype"],
    "confidence_score": 0.95,
    "classification_notes": "comprehensive summary including distinguishing features, unique markings, condition assessment, technology features, and any special characteristics observed across all images",
    "condition_assessment": "excellent/very_good/good/fair/poor",
    "estimated_price_range": "$150-200",
    "features": [
        "adjustable": true,
        "forged": false,
        "cavity_back": true,
        "face_insert": false,
        "weight_ports": true,
        "alignment_aids": false,
        "special_technology": "brief_description"
    ]]
    
    Use proper JSON syntax with curly braces in your response, not square brackets. Grade condition 1-10 scale. Examine all angles for comprehensive analysis. Extract maximum data from all images provided.', 
        TO_FILE('@IMG_STAGE', 'W1.JPG'),
        TO_FILE('@IMG_STAGE', 'W2.JPG'),
        TO_FILE('@IMG_STAGE', 'W3.JPG'),
        TO_FILE('@IMG_STAGE', 'W4.JPG'),
        TO_FILE('@IMG_STAGE', 'W5.JPG'),
        TO_FILE('@IMG_STAGE', 'W6.JPG'),
        TO_FILE('@IMG_STAGE', 'W7.JPG'),
        TO_FILE('@IMG_STAGE', 'W8.JPG'))
);

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'claude-4-sonnet',
    PROMPT('Analyze image {0}, image {1}, image {2}, and image {3} of the same golf club. Identify brand, model, type, loft, and grade condition 1-10. Return JSON with club_type, brand, model, loft, overall_grade, condition_assessment, estimated_price_range.', TO_FILE('@IMG_STAGE', 'Opus_1.jpg'), 
TO_FILE('@IMG_STAGE', 'Opus_2.jpg'), 
TO_FILE('@IMG_STAGE', 'Opus_3.jpg'), 
TO_FILE('@IMG_STAGE', 'Opus_4.jpg'))
)

In [ ]:
SELECT 
    SERVICE_TYPE,
    DATE_TRUNC('MONTH', END_TIME) as MONTH,
    SUM(CREDITS_USED) as TOTAL_CREDITS,
    COUNT(DISTINCT DATE(END_TIME)) as DAYS_WITH_USAGE,
    ROUND(SUM(CREDITS_USED) / COUNT(DISTINCT DATE(END_TIME)), 2) as AVG_DAILY_CREDITS
FROM SNOWFLAKE.ACCOUNT_USAGE.METERING_HISTORY
WHERE END_TIME >= DATEADD('MONTH', -1, CURRENT_DATE)
GROUP BY 1, 2
ORDER BY 3 DESC;